### import

In [1]:
# --- iPython Config --- #
%load_ext autoreload
%autoreload 2

In [2]:
# --- System and Path --- #
import os
import sys
REPO_PATH = os.path.abspath(os.path.join('..')) # depend on specific directory structure
if REPO_PATH not in sys.path:
    sys.path.append(REPO_PATH)
import warnings
warnings.filterwarnings("ignore")

# --- Data Manipulation --- #
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm # Progress bar

# --- Financial Data --- #
import yfinance as yf

# --- Modules --- #
from src import *

# --- Visualization --- #
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
# Vizro import at the end # avoid dependency

In [3]:
# Read Data
df_US = pd.read_csv(REPO_PATH + "/data/private/csv/20240815_2255-DimeUSStock.csv")
# df_MutualFunds = pd.read_csv(REPO_PATH + "/data/private/csv/20240816-DimeMutualFunds.csv")

# Preprocessing

In [4]:
def preprocess_DimeUSStock(df):

    df["Date"] = df["Completion Date"]
    df.drop(columns=["Submission Date", "Completion Date"], inplace=True)
    df["Date"] = pd.to_datetime(df["Date"])
    df["Date"] = pd.to_datetime(df["Date"].dt.date)

    # Drop Features
    df.drop(
        columns=[
            "Market",
            "Status",
            "Total Amount",
            "Stock Amount",
            "Exchange Rate USD:THB (THB)",
            "Commission Fee",
            "VAT",
            "SEC Fee",
            "TAF Fee",
            "Order Type",
        ],
        inplace=True,
    )

    # DataTypes
    for col in df.columns:
        if df[col].dtypes == "object":
            df[col] = df[col].str.replace(",", "")
    df = df.astype(
        {
            "Executed Price (USD)": "float",
            "Total Amount (USD)": "float",
            "Shares": "float",
        }
    )

    df["Volume"] = df["Shares"] * df["Position"].apply(
        lambda x: 1 if x == "Buy" else -1
    )
    # df["Total Amount (USD)"] = df["Total Amount (USD)"] * df["Position"].apply(lambda x: 1 if x == "Buy" else -1)
    df.drop(columns=["Shares", "Position"], inplace=True)

    return df


df_transaction = preprocess_DimeUSStock(df_US.copy())

In [5]:
# NVDA Stock Split
# 2024-06-10, 10:1

df_transaction.loc[
    (df_transaction["Ticker"] == "NVDA") & (df_transaction["Date"] < "2024-06-10"),
    "Volume",
] = (
    df_transaction["Volume"] * 10
)

df_transaction.loc[
    (df_transaction["Ticker"] == "NVDA") & (df_transaction["Date"] < "2024-06-10"),
    "Executed Price (USD)",
] = (
    df_transaction["Executed Price (USD)"] / 10
)

In [6]:
df_portfolio = (
    df_transaction.groupby(["Ticker", "Date"])
    .agg({"Executed Price (USD)": "mean", "Total Amount (USD)": "sum", "Volume": "sum"})
    .reset_index()
)

df_portfolio["Cumulative Volume"] = df_portfolio.groupby("Ticker")["Volume"].cumsum()
df_portfolio["Cumulative Volume"] = df_portfolio["Cumulative Volume"].apply(
    lambda x: 0 if abs(x) < 1e-6 else x
)

# Cumulative Cost (USD)
# is the total amount(cost included fee(discounted in volume)) of money spent on the stock
df_portfolio["Cumulative Cost (USD)"] = (
    df_portfolio.groupby("Ticker")
    .apply(lambda x: (x["Executed Price (USD)"] * x["Volume"]).cumsum())
    .reset_index(drop=True)
)
df_portfolio["Cumulative Cost (USD)"] = df_portfolio["Cumulative Cost (USD)"].apply(
    lambda x: 0 if abs(x) < 0.1 else x
)

df_portfolio["Average Cost (USD)"] = (
    df_portfolio.groupby("Ticker")
    .apply(
        lambda x: (x["Executed Price (USD)"] * x["Volume"]).cumsum()
        / x["Cumulative Volume"]
    )
    .reset_index(drop=True)
)

df_portfolio.drop(
    columns=["Volume", "Executed Price (USD)", "Total Amount (USD)"], inplace=True
)

In [7]:
# Reindex for every
TODAY = datetime.today().strftime("%Y-%m-%d")
all_dates = pd.date_range(start=df_portfolio["Date"].min(), end=TODAY)

df_portfolio = (
    df_portfolio.set_index("Date")
    .groupby("Ticker")
    .apply(lambda x: x.reindex(all_dates).ffill(axis=0))
    .drop(columns="Ticker")
    .reset_index()
    .rename(columns={"level_1": "Date"})
)

## yfinance

In [ ]:
tickers = set(df_portfolio["Ticker"].unique())
tickers.add("THB=X")

DataTerminal = DataTerminal()
df_yf = DataTerminal.fetch_data(tickers)

In [ ]:
# Exchange Rate USD:THB

usdthb = df_yf.query('Ticker == "THB=X"')
usdthb.drop_duplicates(subset="Date", inplace=True)
usdthb.set_index("Date", inplace=True)

# merge
df_portfolio["USDTHB (THB)"] = df_portfolio["Date"].map(usdthb["Adj Close"])
usdthb_ffill = df_portfolio.groupby("Date")["USDTHB (THB)"].mean().ffill()
df_portfolio["USDTHB (THB)"] = df_portfolio["Date"].map(usdthb_ffill)
df_portfolio.dropna(inplace=True)

# convert to THB
df_portfolio["Cumulative Cost (THB)"] = (
    df_portfolio["Cumulative Cost (USD)"] * df_portfolio["USDTHB (THB)"]
)
df_portfolio["Average Cost (THB)"] = (
    df_portfolio["Average Cost (USD)"] * df_portfolio["USDTHB (THB)"]
)

In [ ]:
# merge Market Price
df_portfolio = df_portfolio.merge(
    df_yf.query('Ticker != "THB=X"'),
    how="left",
    left_on=["Date", "Ticker"],
    right_on=["Date", "Ticker"],
)

mktpriceffill = df_portfolio.groupby(["Ticker", "Date"])["Adj Close"].mean().ffill().reset_index()
mktpriceffill.rename(columns={"Adj Close": "Market Price (USD)"}, inplace=True)

df_portfolio = df_portfolio.merge(
    mktpriceffill, how="left", left_on=["Ticker", "Date"], right_on=["Ticker", "Date"]
)
df_portfolio.drop(columns=["Adj Close"], inplace=True)

## Benchmark

| Name | yf-Ticker |
| --- | --- |
| S&P 500 | ^GSPC |
| Dow Jones Industrial Average | ^DJI |
| NASDAQ 100 | ^NDX |
| Gold Dec 24 | GC=F |
| SET Index | ^SET.BK |
| USD/THB | THB=X |
| T-Note 10Y | ^TNX |

In [ ]:
benchmark_dict = {
    "^GSPC": "S&P 500",
    "^DJI": "Dow Jones",
    "^NDX": "Nasdaq",
    "GC=F": "Gold",
    "^SET.BK": "SET",
    "THB=X": "USDTHB",
    "^NDXT": "NDQ100Tech",
    "^TNX": "T-Note 10Y",
}
tickers = benchmark_dict.keys()
df_benchmark = DataTerminal.fetch_data(tickers)
df_benchmark["Ticker"] = df_benchmark["Ticker"].replace(benchmark_dict)

In [ ]:
df_benchmark['Return_1d (%)'] = df_benchmark.groupby('Ticker')['Adj Close'].pct_change(1) * 100

first_day = df_portfolio['Date'].min()
df_benchmark = df_benchmark[df_benchmark['Date'] >= first_day]

df_benchmark['CumReturn (%)'] = df_benchmark.groupby('Ticker')['Return_1d (%)'].cumsum()

## Calculation

In [ ]:
# Portfolio Cost (THB)
portfolio_cost = df_portfolio.groupby("Date")["Cumulative Cost (THB)"].sum()
portfolio_cost.name = "Portfolio Cost (THB)"
df_portfolio = df_portfolio.merge(
    portfolio_cost, how="left", left_on="Date", right_on="Date"
)

# Portfolio Value
df_portfolio["Market Value (USD)"] = (
    df_portfolio["Market Price (USD)"] * df_portfolio["Cumulative Volume"]
)
portfolio_value = df_portfolio.groupby("Date")["Market Value (USD)"].sum()
portfolio_value.name = "Portfolio Value (USD)"
df_portfolio = df_portfolio.merge(
    portfolio_value, how="left", left_on="Date", right_on="Date"
)
df_portfolio["Portfolio Value (THB)"] = (
    df_portfolio["Portfolio Value (USD)"] * df_portfolio["USDTHB (THB)"]
)

In [14]:
# Return
# portfolio
df_portfolio["Portfolio CumReturn (%)"] = (
    (df_portfolio["Portfolio Value (THB)"] - df_portfolio["Portfolio Cost (THB)"])
    / df_portfolio["Portfolio Cost (THB)"]
    * 100
)
df_portfolio["Portfolio Return_1d (%)"] = df_portfolio["Date"].map(
    df_portfolio.groupby("Date")["Portfolio CumReturn (%)"].mean().diff(1)
)
df_portfolio["Portfolio CumReturn (THB)"] = (
    df_portfolio["Portfolio Value (THB)"] - df_portfolio["Portfolio Cost (THB)"]
)

# asset
df_portfolio["Asset CumReturn (%)"] = (
    (df_portfolio["Market Price (USD)"] - df_portfolio["Average Cost (USD)"])
    / df_portfolio["Average Cost (USD)"]
    * 100
)
df_portfolio["Asset CumReturn (THB)"] = (
    (df_portfolio["Market Price (USD)"] - df_portfolio["Average Cost (USD)"])
    * df_portfolio["Cumulative Volume"]
    * df_portfolio["USDTHB (THB)"]
)

In [ ]:
df_portfolio

## Sharpe Ratio

$$Sharpe\ ratio = \frac{R_p - R_f}{\sigma_p}$$

In [ ]:
# Risk Free Rate # CBOE Interest Rate 10 Year T No
risk_free_rate = DataTerminal.fetch_data(["^TNX"]) # percent
# no data on market closing day

risk_free_rate.drop_duplicates(subset="Date", inplace=True)
risk_free_rate.set_index("Date", inplace=True)
risk_free_rate.drop(columns=["Ticker"], inplace=True)
risk_free_rate.rename(columns={"Adj Close": "Annual Rate (%)"}, inplace=True)

risk_free_rate['Daily Rate (%)'] = risk_free_rate['Annual Rate (%)'] / 252

In [17]:
df_sharpe = df_portfolio.groupby("Date")["Portfolio Return_1d (%)"].mean().to_frame()
df_sharpe['ExcessReturn_1d (%)'] = df_sharpe['Portfolio Return_1d (%)'] - risk_free_rate['Daily Rate (%)']

df_sharpe['Sharpe Ratio'] = df_sharpe['ExcessReturn_1d (%)'].expanding().sum() / df_sharpe['ExcessReturn_1d (%)'].expanding().std()

In [ ]:
# Sharpe ratio - Line plot
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df_sharpe.index,
        y=df_sharpe["Sharpe Ratio"],
        mode="lines",
        name="Sharpe Ratio",
    )
)
# MA3
fig.add_trace(
    go.Scatter(
        x=df_sharpe.index,
        y=df_sharpe["Sharpe Ratio"].rolling(window=21*3).mean(),
        mode="lines",
        name="MA3",
    )
)
# yellow h-line at y=1
fig.add_shape(
    dict(
        type="line",
        xref="paper",
        yref="y",
        x0=0,
        y0=1,
        x1=1,
        y1=1,
        line=dict(color="yellow", width=2, dash="dash"),
    )
)
# red h-line at y=0
fig.add_shape(
    dict(
        type="line",
        xref="paper",
        yref="y",
        x0=0,
        y0=0,
        x1=1,
        y1=0,
        line=dict(color="red", width=2, dash="dash"),
    )
)

# # green band at y=0 to y=+INF
# fig.add_shape(
#     dict(
#         type="rect",
#         xref="paper",
#         yref="y",
#         x0=0,
#         y0=0,
#         x1=1,
#         y1=df_sharpe["Sharpe Ratio"].max(),
#         fillcolor="green",
#         opacity=0.15,
#         layer="below",
#         line_width=0,
#     )
# )
# # yellow band at y=1 to y=0
# fig.add_shape(
#     dict(
#         type="rect",
#         xref="paper",
#         yref="y",
#         x0=0,
#         y0=0,
#         x1=1,
#         y1=1,
#         fillcolor="yellow",
#         opacity=0.15,
#         layer="below",
#         line_width=0,
#     )
# )
# # red band at y=0 to y=-INF
# fig.add_shape(
#     dict(
#         type="rect",
#         xref="paper",
#         yref="y",
#         x0=0,
#         y0=0,
#         x1=1,
#         y1=df_sharpe["Sharpe Ratio"].min(),
#         fillcolor="red",
#         opacity=0.15,
#         layer="below",
#         line_width=0,
#     )
# )
fig.update_layout(
    title="Sharpe Ratio",
    xaxis_title="Date",
    yaxis_title="Ratio",
    template="plotly_dark",
)
fig.show()

# Visualization

In [ ]:
# stacked bar plot
fig = px.bar(
    df_portfolio, x="Date", y="Cumulative Cost (THB)", color="Ticker", barmode="stack"
)
fig.update_layout(
    title="Cumulative Cost (THB)",
    xaxis_title="Date",
    yaxis_title="Cumulative Cost (THB)",
    template="plotly_dark",
)
fig.show()

In [ ]:
# tree map for lastest date
df = df_portfolio[df_portfolio["Date"] == df_portfolio["Date"].max()]
fig = px.treemap(
    df,
    path=["Ticker"],
    values="Cumulative Cost (THB)",
    title="Cumulative Cost (THB)",
    template="plotly_dark",
)
fig.show()

In [ ]:
# Grouping and calculating the mean portfolio return by date
df = df_portfolio.groupby("Date")["Portfolio CumReturn (%)"].mean() # series

fig = go.Figure()

# Adding area color with conditional coloring
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df,
        mode="lines",
        name="Portfolio Return (%)",
        fill="tozeroy",  # Fills the area under the curve
        fillcolor=str(np.where(df[-1] >= 0, "rgba(0, 255, 0, 0.2)", "rgba(255, 0, 0, 0.2)")),  # Green color for positive values, red color for negative values
        line=dict(color="green"),  # Green line for positive values
    )
)

# Updating the layout
fig.update_layout(
    title="Portfolio Cumulative Return (%)",
    xaxis_title="Date",
    yaxis_title="Return (%)",
    template="plotly_dark",
)

# Displaying the plot
fig.show()

In [ ]:
# line plot Portfolio Cost (THB) vs Portfolio Value (THB)

df = df_portfolio.groupby("Date")[
    ["Portfolio Cost (THB)", "Portfolio Value (THB)"]
].mean()

fig = go.Figure()

# Adding the Portfolio Cost (THB) line
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df["Portfolio Cost (THB)"],
        mode="lines",
        name="Portfolio Cost (THB)",
        line=dict(color="blue"),
    )
)

# Adding the Portfolio Value (THB) line
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df["Portfolio Value (THB)"],
        mode="lines",
        name="Portfolio Value (THB)",
        line=dict(color="red"),
    )
)

# Updating the layout
fig.update_layout(
    title="Portfolio Cost (THB) vs Portfolio Value (THB)",
    xaxis_title="Date",
    yaxis_title="Amount (THB)",
    template="plotly_dark",
)

# Displaying the plot
fig.show()

In [ ]:
# line plot Portfolio CumReturn (%) vs df_benchmark

df = df_portfolio.groupby("Date")["Portfolio CumReturn (%)"].mean()

fig = go.Figure()

# Adding the Portfolio CumReturn (%) line

fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df,
        mode="lines",
        name="Portfolio CumReturn (%)",
        line=dict(color="blue"),
    )
)

# Adding the Benchmark CumReturn (%) line
for ticker in df_benchmark["Ticker"].unique():
    df = df_benchmark[df_benchmark["Ticker"] == ticker]
    fig.add_trace(
        go.Scatter(
            x=df["Date"],
            y=df["CumReturn (%)"],
            mode="lines",
            name=ticker,
        )
    )

# Updating the layout
fig.update_layout(
    title="Cumulative Return (%): Portfolio vs Benchmark",
    xaxis_title="Date",
    yaxis_title="Return (%)",
    template="plotly_dark",
)

# Displaying the plot
fig.show()

## Vizro

In [24]:
# import vizro.plotly.express as px
# from vizro import Vizro
# import vizro.models as vm

# df = df_portfolio

# page = vm.Page(
#     title="Portfolio dashboard",
#     components=[
#         vm.Graph(
#             id="treemap",
#             figure=px.treemap(
#                 df,
#                 path=["Ticker"],
#                 values="Cumulative Cost (THB)",
#                 title="Portfolio Value",
#             ),
#         ),
#         vm.Graph(
#             id="stack_bar",
#             figure=px.bar(
#                 df, x="Date", y="Cumulative Cost (THB)", color="Ticker", barmode="stack"
#             ),
#         ),
#         vm.Graph(

#         )
#     ],
#     controls=[
#         # vm.Filter(column="species", selector=vm.Dropdown(value=["ALL"])),
#     ],
# )

# dashboard = vm.Dashboard(pages=[page])

# Vizro().build(dashboard).run()